### Korean Security Market

#### 1. Data Import

**1.1 KOSPI 상장기업 불러오기**

Notion Database에 있는 `stock_list.csv`는 한국 KOSPI, KOSDAQ, KONEX에 상장되어 있는 모든 기업의 종목 정보를 저장한 파일이다. Code에서 0으로 끝나는 종목만이 거래가 원활하기 때문에 필터링 과정이 필요하다

종목코드가 0으로 끝나는 ticker만을 필터링하여 상위 200개의 종목의 티커를 list형태로 저장하라

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import warnings 
warnings.filterwarnings('ignore')

In [ ]:
# Write your code here!

**1.2 가격 정보 불러오기**

아래의 library를 이용하여 2020년 1월 1일부터 2023년 11월 23일까지의 200개 종목에 대한 종가 데이터를 불러와 dataframe에 저장하라

In [ ]:
import pandas_datareader as pdr
import datetime

data = pd.DataFrame()

start_date = datetime.datetime(2020, 1, 1)
end_date = datetime.datetime(2023, 11, 23)

# Write your code here!

**1.3 수익률 변환**

log차분을 수행하여 안정적인 시계열 데이터로 변환하라. seaborn의 heatmap함수를 사용하여 자산간 상관성을 확인한다

In [ ]:
import seaborn as sns

# Write your code here!

#### 2. Principal Components Analysis

sklearn의 PCA를 활용하여 차원 축소를 수행하라. `n_components = 10`으로 지정한다. clustermap을 확인해 보자. 어떤 결과가 나오는가? clustering은 잘 수행이 되었는가?

In [ ]:
from sklearn.decomposition import PCA

# Write your code here!

#### 3. K means clustering

sklearn의 KMeans를 사용하여 군집화를 수행하라. 군집화의 개수는 10으로 지정한다. 수행한 결과를 각각 mean-variance 평면과 시간에 따른 수익률 plot에 시각화해 보자. 어떤 결과를 얻을 수 있는가?

In [ ]:
from sklearn.cluster import KMeans

# Write your code here!

#### 4. Hierarchical PCA

자산 200개에 대한 Dendogram을 출력하고 clustermap을 확인해 보자. distance는 $\sqrt{\frac{1}{2}(1 - \rho_{i,j})}$로 정의한다

In [ ]:
from scipy.cluster.hierarchy import dendrogram, linkage
from scipy.spatial.distance import squareform

# Write your code here!

#### 5. Quasi Diagonalization

4번의 결과에 이어 아래의 함수를 사용해 준 대각화를 시행해 보고 결과를 출력하라

In [ ]:
def QuasiDiag(link: np.ndarray) -> list:
    link = link.astype(int)
    sortIx = pd.Series([link[-1, 0], link[-1, 1]])
    numItems = link[-1, 3]    # number of original items
    while sortIx.max() >= numItems:
        sortIx.index = range(0, sortIx.shape[0] * 2, 2)    # make space
        df0 = sortIx[sortIx >= numItems]    # find clusters
        i = df0.index
        j = df0.values - numItems
        sortIx[i] = link[j, 0]    # item 1
        df0 = pd.Series(link[j, 1], index=i+1)
        sortIx = pd.concat([sortIx, df0])    # item 2
        sortIx = sortIx.sort_index()    # re-sort
        sortIx.index = range(sortIx.shape[0])    # re-index
    lst =  sortIx.tolist()
    return lst

In [ ]:
# Write your code here!

#### 6. IPCA Allocation

5번의 추정된 결과와 아래의 함수를 사용하여 각 자산별 가중치를 계산하라

In [ ]:
def InversePortfolio(cov, **kargs) : 
    ivp = 1 / np.diag(cov)
    ivp /= ivp.sum()
    return ivp

def ClusterVar(cov, cItems) :
    cov_ = pd.DataFrame(cov).loc[cItems, cItems] # 행렬 분할
    w_ = InversePortfolio(cov_).reshape(-1, 1)
    cVar = np.dot(np.dot(w_.T, cov_), w_)[0, 0]
    return cVar

def RecBipart(cov: np.ndarray, sortIx: list) -> pd.Series:
    w = pd.Series([1] * len(sortIx), index=sortIx)
    cItems = [sortIx]    # 모든 아이템을 하나의 군집으로 초기화
    while len(cItems) > 0:
        cItems = [i[int(j): int(k)] for i in cItems
                  for j, k in ((0, len(i) / 2), (len(i) / 2, len(i))) if len(i) > 1]    # bi-section
        for i in range(0, len(cItems), 2):    # parse in pairs
            cItems0 = cItems[i]    # cluster 1
            cItems1 = cItems[i+1]    # cluster 2
            cVar0 = ClusterVar(cov, cItems0)
            cVar1 = ClusterVar(cov, cItems1)
            alpha = 1 - cVar0 / (cVar0 + cVar1)
            w[cItems0] *= alpha    # weight 1
            w[cItems1] *= 1 - alpha    # weight 2
    return w

In [ ]:
# Write your code here!